In [1]:
import requests
from os import path
import csv
import numpy as np
import sys
import os
import codecs
import re
from IPython.display import Markdown
sys.path.insert(0, os.path.abspath('../..'))
from probability import FrequencyTable

%matplotlib inline

fname = 'data/Moby.Dick.txt'
if not path.exists('data'):
    os.mkdir('data')
if not path.exists(fname):
    url = 'http://www.gutenberg.org/files/2701/2701-0.txt'
    r = requests.get(url, allow_redirects=True)    
    file = codecs.open(fname, 'w', "utf-8")
    file.write(r.content.decode('utf-8'))


In [2]:
# Load the text
file = codecs.open(fname, 'r', 'utf-8')
# Remove all '\r' and '\n'
lines = [l.replace('\r\n','') for l in file.readlines()]
file.close()

In [3]:
single_line = np.r_[lines]

In [4]:
def replace_char(line):    
    # convert more-than-one spaces to one
    line = re.sub(" +", " " , line) 
    # remove punctuations
    line = re.sub("[,;:’\*\#\[\]()!?“”_/\-]", "" , line) 
    # lower case
    line = line.lower()
    # replace . by STOP_WORD    
    line = line.replace(".", " STOP_WORD ")    
    return line
#np.flatiter()
words = [w for line in [replace_char(line).split(" ") for line in single_line if len(line) > 0] 
           for w in line if len(w) > 0]
dist = FrequencyTable(words)

In [5]:
r = {k:v for k, v in dist.most_common(10)}
Markdown(FrequencyTable(r).to_table(sort=True))

|X1       |     |
|---------|-----|
|      the|14594|
|STOP_WORD|8188 |
|       of|6711 |
|      and|6448 |
|        a|4705 |
|       to|4659 |
|       in|4210 |
|     that|2951 |
|      his|2522 |
|       it|2383 |


In [6]:
from itertools import tee

def to_tuple(iterable, n=2):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    iterables = tee(iterable, n)
    for i in range(n):
        for j in range(i,n):
            next(iterables[j], None)
        
    return zip(*tuple(iterables))

two_words = to_tuple(words)
dist2 = FrequencyTable([w2 for w2 in two_words if w2[0] != "STOP_WORD"])

In [7]:
r = {k:v for k, v in dist2.most_common(10)}
Markdown(FrequencyTable(r).to_table(sort=True))

|X1  |X2 |    |
|----|---|----|
|  of|the|1911|
|  in|the|1189|
|  to|the|743 |
|from|the|444 |
| and|the|372 |
|  of|his|371 |
|  on|the|356 |
|  of|  a|338 |
|  at|the|332 |
|  to| be|329 |


In [8]:
three_words = to_tuple(words, 3)
dist3 = FrequencyTable([w3 for w3 in three_words if w3[0] != "STOP_WORD" and w3[1] != "STOP_WORD"])

In [9]:
r = {k:v for k, v in dist3.most_common(20)}
Markdown(FrequencyTable(r).to_table(sort=True))

|X1  |X2   |X3       |  |
|----|-----|---------|--|
| the|sperm|    whale|86|
|  of|  the|    whale|78|
| the|white|    whale|71|
| one|   of|      the|64|
| out|   of|      the|57|
|  of|  the|      sea|57|
|part|   of|      the|53|
|   a| sort|       of|51|
| the|  sea|STOP_WORD|45|
|  of|  the|    sperm|43|
| the|whale|STOP_WORD|35|
|  it|  was|        a|33|
|  in|  the|      sea|32|
| the|sperm|   whales|31|
| for|    a|   moment|29|
|  of|  the|     boat|29|
|  it|   is|        a|29|
|  of|  the|     ship|28|
|  of|  the|   whales|28|
|  to|  the|     deck|27|


In [10]:
cond = dist3.condition_on("X1")

In [11]:
dist4 = cond["sea"]

In [12]:
for word2, word3 in list(dist4)[:10]:
    print(word2, word3, "\t\t%.5f" % dist4.prob(X2=word2,X3=word3))

a bouncing 		0.00324
adding largely 		0.00324
after them 		0.00324
again in 		0.00324
air as 		0.00324
all whose 		0.00324
almost perpetually 		0.00324
alternate with 		0.00324
among waves 		0.00324
and a 		0.00324


In [13]:
reduced_dist = dist3.reduce(X1="sea")

In [15]:
r = {k:v for k, v in FrequencyTable(reduced_dist).most_common(20)}
Markdown(FrequencyTable(r).to_table(sort=True))

|X1         |X2         | |
|-----------|-----------|-|
|        and|        the|6|
|         as|          a|5|
|         in|          a|3|
|         as|         if|3|
|       such|          a|2|
|         on|        the|2|
|       even|         as|2|
|disappeared|         in|2|
|         by|        the|2|
|         as|        the|2|
|        and|     though|2|
|        and|         in|2|
|        and|    finally|2|
|     almost|perpetually|1|
|        all|      whose|1|
|        air|         as|1|
|      again|         in|1|
|      after|       them|1|
|     adding|    largely|1|
|          a|   bouncing|1|


In [16]:
marginalised_dist = dist3.marginal("X2", "X3")

In [17]:
r = {k:v for k, v in marginalised_dist.most_common(20)}
Markdown(FrequencyTable(r).to_table(sort=True))

|X1  |     |
|----|-----|
| the|14594|
|  of|6704 |
| and|6448 |
|   a|4691 |
|  to|4642 |
|  in|4197 |
|that|2926 |
| his|2511 |
|  it|2190 |
|   i|1906 |
| but|1775 |
|with|1763 |
|  he|1742 |
|  as|1730 |
|  is|1720 |
| was|1630 |
| for|1621 |
| all|1457 |
|this|1396 |
|  at|1322 |


In [10]:
dist_ten_1 = FrequencyTable(words[:10000], ["X1"])
dist_ten_2 = FrequencyTable(words[:10000], ["X2"])

In [11]:
%prun prod_1 = dist_ten_1 * dist_ten_2

In [19]:
%prun prod_1 = dist_ten_1.product_multi_proc(dist_ten_2, 4)

In [14]:
len(prod_1.keys())

166435801

In [21]:
len(dist_ten_1.keys())

12901

In [12]:
%prun p2 = prod_1.marginal("X1")

In [13]:
print(p2.summary())

Frequency Table 
Column names:'['X2']'
total:100000000
normalised:False



In [14]:
len(prod_1)

8898289

In [15]:
len(p2)

2983